In [1]:
import os
import sys
import json

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
# root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
sys.path.append(root)


# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from experiment.ember_dataset.ember_model import EmberModel
from data_preprocessing import *

2023-08-12 15:02:59.696882: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-12 15:02:59.736058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 15:03:00.270253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf

In [3]:

with open('conf.json', 'r') as json_file:
    config = json.load(json_file)


In [5]:
X, y= load_data(f'./../ember_data/chunk_0.pickle')

X_val, y_val = load_data(f'./../ember_data/test_set.pickle')

X = np.array(X)
y = np.array(y)
print('X shape:', X.shape, '-- y shape:', y.shape)
print('X val shape:', X_val.shape, '-- y val shape:', y_val.shape)

data_size = len(X)
data_size

X shape: (60028, 2381) -- y shape: (60028,)
X val shape: (200000, 2381) -- y val shape: (200000,)


60028

In [8]:
X.shape[1]

2381

In [6]:
train_ds = create_dataset(X, y)
test_ds = create_dataset(X_val, y_val)
type(train_ds), type(test_ds)


2023-08-12 15:04:22.024852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-12 15:04:22.237502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-12 15:04:22.237548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-12 15:04:22.254839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-12 15:04:22.254899: I tensorflow/compile

(tensorflow.python.data.ops.prefetch_op._PrefetchDataset,
 tensorflow.python.data.ops.prefetch_op._PrefetchDataset)

In [10]:

epoch = 20
batch_size = 128

learning_rate_config = config.get('training_params').get('learning_rate_config', {}) 
if learning_rate_config == {}:
    learning_rate_config['fix_lr'] = True
    learning_rate_config['initial_lr'] = 0.1
    learning_rate_config['min_lr'] = 0.001
    learning_rate_config['decay_steps'] = data_size * epoch / batch_size

In [13]:

# Define model
model = EmberModel(input_features= X.shape[1], output_features= 1, lr_config= learning_rate_config)


lr config in the resnet model
{'fix_lr': True, 'initial_lr': 0.1, 'min_lr': 0.001, 'decay_steps': 9379.375}
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2381)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 2381, 8)      2056        ['input_3[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (None, 158, 128)     15488       ['embedding_2[0][0]']            
                                                                                                  
 conv1d_5 (Conv1D)              (None, 158, 128)     15488       ['embedding_2[0][0

In [14]:
# Define framework
tensorflow_framework = TensorflowFramework(model=model, 
                                           data_size= data_size, 
                                           train_ds= train_ds, 
                                           test_ds= test_ds, 
                                           config=config)


In [15]:
epoch

20

In [16]:
for epoch in range(epoch):
    print(f"Enter epoch {epoch + 1}, learning rate = {tensorflow_framework.get_learning_rate()}")
    tensorflow_framework.model.train_loss.reset_states()
    tensorflow_framework.model.train_performance.reset_states()
    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for images, labels in tensorflow_framework.train_ds:
        train_acc, train_loss= tensorflow_framework.fit(images, labels)

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("*" * 20)
    print("Epoch {} - Train Acc: {:.2f} -- Train Loss {} Test Acc {:.2f}  Test Loss {}".format(epoch+1,
                                                                                       train_acc * 100,
                                                                                       train_loss,
                                                                                       test_acc * 100,
                                                                                       test_loss))
    print("*" * 20)


Enter epoch 1, learning rate = 0.10000000149011612


2023-08-12 15:07:50.963580: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60028,2381]
	 [[{{node Placeholder/_0}}]]
2023-08-12 15:07:50.964113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60028,2381]
	 [[{{node Placeholder/_0}}]]
2023-08-12 15:08:04.424888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


In [ ]:
# save weights
save_location = "weights2.pkl"
weights = model.get_weights()

with open(save_location, 'wb') as f:
    import pickle
    pickle.dump(weights, f)